https://medium.com/@ramanbazhanau/mastering-sqlalchemy-a-comprehensive-guide-for-python-developers-ddb3d9f2e829

### Creating Your First SQLAlchemy Project

In [21]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# Create an engine that stores data in the local directory's
# sqlalchemy_example.db file.
engine = create_engine('sqlite:///sqlalchemy_example.db')

# Create a declarative base class
Base = declarative_base()

# Define the Book model
class Book(Base):
    __tablename__ = 'books'

    id = Column(Integer, primary_key=True)
    title = Column(String(250), nullable=False)
    author = Column(String(250), nullable=False)
    genre = Column(String(100))

    def __repr__(self):
        return f"<Book(title='{self.title}', author='{self.author}', genre='{self.genre}')>"

# Create all tables in the engine
Base.metadata.create_all(engine)

# Create a session
Session = sessionmaker(bind=engine)
session = Session()

# Add a book
new_book = Book(title='To Kill a Mockingbird', author='Harper Lee', genre='Fiction')
session.add(new_book)
session.commit()

# Query the book
book = session.query(Book).filter_by(title='To Kill a Mockingbird').first()
print(book)

# Close the session
session.close()

<Book(title='To Kill a Mockingbird', author='Harper Lee', genre='Fiction')>


C:\Users\vainer.ag\AppData\Local\Temp\ipykernel_12572\2054638666.py:10: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


### SQLAlchemy Core
#### Understanding the Expression Language
##### 1. Table and Column Representations

In [22]:
from sqlalchemy import Table, Column, Integer, String, MetaData

metadata = MetaData()
users = Table('users', metadata,
    Column('id', Integer, primary_key=True),
    Column('name', String),
    Column('fullname', String)
)

##### 2. SQL Expressions

In [23]:
from sqlalchemy import select

# SELECT * FROM users
stmt = select(users)

# SELECT name, fullname FROM users WHERE name = 'ed'
stmt = select(users.c.name, users.c.fullname).where(users.c.name == 'ed')

##### 3. Joins

In [24]:
addresses = Table('addresses', metadata,
    Column('id', Integer, primary_key=True),
    Column('user_id', Integer),
    Column('email_address', String)
)

# JOIN users and addresses
stmt = select(users, addresses).join(addresses, users.c.id == addresses.c.user_id)

##### 4. Ordering and Grouping

In [25]:
from sqlalchemy import desc

# ORDER BY
stmt = select(users).order_by(desc(users.c.name))

# GROUP BY
from sqlalchemy import func
stmt = select(users.c.name, func.count(addresses.c.id)).join(addresses).group_by(users.c.name)

#### Working with Tables and Schemas
##### 1. Creating Tables

In [26]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///example.db')
metadata.create_all(engine)

##### 2. Reflecting Existing Tables

In [27]:
metadata = MetaData()
users = Table('users', metadata, autoload_with=engine)

##### 3. Altering Tables

In [28]:
from sqlalchemy import Table, Column, Integer, String, MetaData


def upgrade(engine):
    meta = MetaData(bind=engine)
    users = Table('users', meta, autoload=True)
    new_column = Column('email', String(50))
    new_column.create(users)


def downgrade(engine):
    meta = MetaData(bind=engine)
    users = Table('users', meta, autoload=True)
    users.c.email.drop()

#### Executing SQL Queries using Core
##### 1. Using execute()

In [29]:
from sqlalchemy import create_engine, text

engine = create_engine('sqlite:///example.db')

with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM users WHERE name=:name"), {"name": "ed"})
    for row in result:
        print(row)

(1, 'ed', 'Ed Jones')


##### 2. Using Compiled Statements

In [30]:
stmt = select(users).where(users.c.name == 'ed')
with engine.connect() as conn:
    result = conn.execute(stmt)
    for row in result:
        print(row)

(1, 'ed', 'Ed Jones')


##### 3. Inserting Data

In [31]:
from sqlalchemy import insert

stmt = insert(users).values(name='ed', fullname='Ed Jones')
with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

##### 4. Updating Data

In [32]:
from sqlalchemy import update

stmt = update(users).where(users.c.name == 'ed').values(fullname='Edward Jones')
with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

##### 5. Deleting Data

In [33]:
from sqlalchemy import delete

stmt = delete(users).where(users.c.name == 'ed')
with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

##### 6. Transactions

In [34]:
with engine.begin() as conn:
    conn.execute(insert(users).values(name='ed', fullname='Ed Jones'))
    conn.execute(insert(users).values(name='wendy', fullname='Wendy Williams'))
    # Automatically commits if no exceptions are raised

### SQLAlchemy ORM Basics
#### Defining Models and Relationships
##### 1. Basic Model Definition

In [35]:
from sqlalchemy import Column, Integer, String, create_engine
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()


class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    email = Column(String)

    def __repr__(self):
        return f"<User(name='{self.name}', fullname='{self.fullname}', email='{self.email}')>"


engine = create_engine('sqlite:///example.db')
Base.metadata.create_all(engine)

C:\Users\vainer.ag\AppData\Local\Temp\ipykernel_12572\2394693481.py:4: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


##### 2. Defining Relationships

In [36]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship


class Address(Base):
    __tablename__ = 'addresses'

    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'))

    user = relationship("User", back_populates="addresses")


User.addresses = relationship("Address", order_by=Address.id, back_populates="user")

#### CRUD Operations with ORM
##### 1. Create (Insert):

In [ ]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

new_user = User(name='ed', fullname='Ed Jones', email='ed@example.com')
session.add(new_user)
session.commit()

#### 2. Read (Select)

In [ ]:
# Fetch all users
users = session.query(User).all()

# Fetch a specific user
user = session.query(User).filter_by(name='ed').first()

##### 3. Update

In [ ]:
user = session.query(User).filter_by(name='ed').first()
user.fullname = 'Edward Jones'
session.commit()

##### 4. Delete

In [ ]:
user = session.query(User).filter_by(name='ed').first()
session.delete(user)
session.commit()

##### 1. Basic Queries:

In [ ]:
# Select all users
users = session.query(User).all()

# Select users with a specific name
users = session.query(User).filter_by(name='ed').all()

# Select users with names starting with 'ed'
users = session.query(User).filter(User.name.like('ed%')).all()

##### 2. Ordering Results:

In [ ]:
users = session.query(User).order_by(User.name).all()

##### 3. Limiting Results:

In [ ]:
users = session.query(User).limit(5).all()

##### 4. Joins:

In [ ]:
results = session.query(User, Address).join(Address).all()

5. Aggregations:

In [ ]:
from sqlalchemy import func

# Count the number of users
user_count = session.query(func.count(User.id)).scalar()

# Get the user with the most addresses
user_with_most_addresses = session.query(User, func.count(Address.id).label('address_count')).\
    join(Address).\
    group_by(User).\
    order_by(func.count(Address.id).desc()).\
    first()

##### 6. Subqueries:

In [ ]:
from sqlalchemy import subquery

# Get users with more than 2 addresses
address_count = session.query(Address.user_id, func.count('*').label('address_count')).\
    group_by(Address.user_id).\
    subquery()

users = session.query(User).\
    join(address_count, User.id == address_count.c.user_id).\
    filter(address_count.c.address_count > 2).\
    all()

##### 7. Eager Loading:

In [ ]:
# Load users and their addresses in one query
users = session.query(User).options(joinedload(User.addresses)).all()
session.close()